<a href="https://colab.research.google.com/github/Kussil/CVX_Rice_project/blob/main/03_Sentiment_Analysis/Sentiment_Analysis_(FinBERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Add Sentiment Analysis

In [1]:
#installs

In [2]:
#imports
import os
from google.colab import userdata
import pandas as pd
import math
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch

### Import Files From GitHub

In [3]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/CVX_Rice_project.git

Cloning into 'CVX_Rice_project'...
remote: Enumerating objects: 1544, done.
remote: Counting objects: 100% (533/533), done.
remote: Compressing objects: 100% (261/261), done.
remote: Total 1544 (delta 290), reused 496 (delta 263), pack-reused 1011
Receiving objects: 100% (1544/1544), 226.89 MiB | 9.63 MiB/s, done.
Resolving deltas: 100% (881/881), done.
Updating files: 100% (1082/1082), done.


In [5]:
invest_df1 = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/SEC_Filings.csv')

df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
df['Article Index'] = df.index
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
df['Date']= pd.to_datetime(df['Date']).dt.date
df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index
0,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0
1,Investment Research,NaN,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,1
2,Investment Research,NaN,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,2
3,Investment Research,NaN,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,3
4,Investment Research,NaN,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,4
...,...,...,...,...,...,...,...,...
10319,Earnings Calls,NaN,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN,10319
10320,Earnings Calls,NaN,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10320
10321,Earnings Calls,NaN,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN,10321
10322,Earnings Calls,NaN,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN,10322


### Chunk Text for FinBERT, < 512 token input

In [ ]:
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')

token_limit = 500

def chunk_contents(x):
  x_str = str(x)
  tokens = tokenizer(x_str)['input_ids']
  token_count = len(tokens)
  chunk_count = math.ceil(token_count / token_limit)
  chunk_list = []
  for i in range(chunk_count):
    chunk_list.append(tokenizer.decode(tokens[i*token_limit:(i+1)*token_limit]))
  return chunk_list

df['Text Chunks'] = df['Article Text'].apply(chunk_contents)
df

Token indices sequence length is longer than the specified maximum sequence length for this model (1816 > 512). Running this sequence through the model will result in indexing errors


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index,Text Chunks
0,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"[[CLS] stock report | may 16, 2024 | nysesymbo..."
1,Investment Research,NaN,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,1,"[[CLS] stock report | may 14, 2024 | nysesymbo..."
2,Investment Research,NaN,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,2,"[[CLS] stock report | may 11, 2024 | nysesymbo..."
3,Investment Research,NaN,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,3,"[[CLS] stock report | may 11, 2024 | nysesymbo..."
4,Investment Research,NaN,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,4,"[[CLS] stock report | may 07, 2024 | nysesymbo..."
...,...,...,...,...,...,...,...,...,...
8599,ProQuest,2.238654e+09,PSX,2019-06-12,"As the Price of Gas Tumbles, Even California I...",Gasoline prices have been falling fast in rece...,https://www.proquest.com/newspapers/as-price-g...,8599,[[CLS] gasoline prices have been falling fast ...
8600,ProQuest,2.239478e+09,PSX,2019-06-10,"Phillips 66, Plains plan pipeline system",CUSHING Phillips 66 and Plains All American Pi...,https://www.proquest.com/newspapers/phillips-6...,8600,[[CLS] cushing phillips 66 and plains all amer...
8601,ProQuest,2.238860e+09,PSX,2019-06-07,Autonomous robot inspects in-service diesel st...,"HOUSTON Phillips 66, Square Robot Inc. and Ver...",https://www.proquest.com/newspapers/autonomous...,8601,"[[CLS] houston phillips 66, square robot inc. ..."
8602,ProQuest,2.233474e+09,PSX,2019-06-03,United States/Mexico: Report: Trump's Tariff o...,US President Donald Trump's threats to tax Mex...,https://www.proquest.com/newspapers/united-sta...,8602,[[CLS] us president donald trump's threats to ...


### Apply FinBERT Sentiment Analysis
Create Neurtal, Positive, Negative matrix for each chunk

In [ ]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
#
#sentiment_checkpoint = 'ProsusAI/finbert'
#softmax = torch.nn.Softmax(dim=1)
#
######### Sentiment Analysis ############################
#finbert = BertForSequenceClassification.from_pretrained(sentiment_checkpoint)#,num_labels=3)
#finbert.to(device)
#tokenizer = BertTokenizer.from_pretrained(sentiment_checkpoint)
#
#labels = {0:'neutral', 1:'positive',2:'negative'}
#
#def finbert_sentiment(chunk_list):
#  with torch.no_grad():
#    content_sentiment_list = []
#    for chunk in chunk_list:
#      inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)
#      inputs.to(device)
#      outputs = finbert(**inputs)[0]
#      softmax_outputs = softmax(outputs)[0]
#      content_sentiment_list.append([softmax_outputs[0].item(), softmax_outputs[1].item(), softmax_outputs[2].item()])
#  return content_sentiment_list
#
#df['Chunk Sentiments'] = df['Text Chunks'].apply(finbert_sentiment)
#df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index,Text Chunks,Chunk Sentiments
0,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"[[CLS] stock report | may 16, 2024 | nysesymbo...","[[0.44454342126846313, 0.04386952891945839, 0...."
1,Investment Research,NaN,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,1,"[[CLS] stock report | may 14, 2024 | nysesymbo...","[[0.11372336745262146, 0.012837883085012436, 0..."
2,Investment Research,NaN,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,2,"[[CLS] stock report | may 11, 2024 | nysesymbo...","[[0.07945890724658966, 0.015997793525457382, 0..."
3,Investment Research,NaN,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,3,"[[CLS] stock report | may 11, 2024 | nysesymbo...","[[0.0931524708867073, 0.013625746592879295, 0...."
4,Investment Research,NaN,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,4,"[[CLS] stock report | may 07, 2024 | nysesymbo...","[[0.14364014565944672, 0.01309792697429657, 0...."
...,...,...,...,...,...,...,...,...,...,...
8599,ProQuest,2.238654e+09,PSX,2019-06-12,"As the Price of Gas Tumbles, Even California I...",Gasoline prices have been falling fast in rece...,https://www.proquest.com/newspapers/as-price-g...,8599,[[CLS] gasoline prices have been falling fast ...,"[[0.023741142824292183, 0.950417160987854, 0.0..."
8600,ProQuest,2.239478e+09,PSX,2019-06-10,"Phillips 66, Plains plan pipeline system",CUSHING Phillips 66 and Plains All American Pi...,https://www.proquest.com/newspapers/phillips-6...,8600,[[CLS] cushing phillips 66 and plains all amer...,"[[0.09723199903964996, 0.011335819028317928, 0..."
8601,ProQuest,2.238860e+09,PSX,2019-06-07,Autonomous robot inspects in-service diesel st...,"HOUSTON Phillips 66, Square Robot Inc. and Ver...",https://www.proquest.com/newspapers/autonomous...,8601,"[[CLS] houston phillips 66, square robot inc. ...","[[0.4957221448421478, 0.008983639068901539, 0...."
8602,ProQuest,2.233474e+09,PSX,2019-06-03,United States/Mexico: Report: Trump's Tariff o...,US President Donald Trump's threats to tax Mex...,https://www.proquest.com/newspapers/united-sta...,8602,[[CLS] us president donald trump's threats to ...,"[[0.17639605700969696, 0.7728658318519592, 0.0..."


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

sentiment_checkpoint = 'ProsusAI/finbert'
softmax = torch.nn.Softmax(dim=1)

######## Sentiment Analysis ############################
finbert = BertForSequenceClassification.from_pretrained(sentiment_checkpoint)#,num_labels=3)
finbert.to(device)
tokenizer = BertTokenizer.from_pretrained(sentiment_checkpoint)

labels = {0:'neutral', 1:'positive',2:'negative'}

def finbert_sentiment(chunk_list):
  with torch.no_grad():
    content_sentiment_list = []
    inputs = tokenizer(chunk_list, return_tensors="pt", padding=True, truncation=True)
    inputs.to(device)
    outputs = finbert(**inputs)[0]
    softmax_outputs = softmax(outputs)
    content_sentiment_list = softmax_outputs.tolist()
    return content_sentiment_list

df['Chunk Sentiments'] = df['Text Chunks'].apply(finbert_sentiment)
df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index,Text Chunks,Chunk Sentiments
0,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"[[CLS] stock report | may 16, 2024 | nysesymbo...","[[0.4445439577102661, 0.04386952891945839, 0.5..."
1,Investment Research,NaN,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,1,"[[CLS] stock report | may 14, 2024 | nysesymbo...","[[0.11372338980436325, 0.012837883085012436, 0..."
2,Investment Research,NaN,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,2,"[[CLS] stock report | may 11, 2024 | nysesymbo...","[[0.07945896685123444, 0.015997786074876785, 0..."
3,Investment Research,NaN,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,3,"[[CLS] stock report | may 11, 2024 | nysesymbo...","[[0.09315253049135208, 0.013625753112137318, 0..."
4,Investment Research,NaN,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,4,"[[CLS] stock report | may 07, 2024 | nysesymbo...","[[0.14364004135131836, 0.013097928836941719, 0..."
...,...,...,...,...,...,...,...,...,...,...
8599,ProQuest,2.238654e+09,PSX,2019-06-12,"As the Price of Gas Tumbles, Even California I...",Gasoline prices have been falling fast in rece...,https://www.proquest.com/newspapers/as-price-g...,8599,[[CLS] gasoline prices have been falling fast ...,"[[0.023741185665130615, 0.950417160987854, 0.0..."
8600,ProQuest,2.239478e+09,PSX,2019-06-10,"Phillips 66, Plains plan pipeline system",CUSHING Phillips 66 and Plains All American Pi...,https://www.proquest.com/newspapers/phillips-6...,8600,[[CLS] cushing phillips 66 and plains all amer...,"[[0.09723199903964996, 0.011335819028317928, 0..."
8601,ProQuest,2.238860e+09,PSX,2019-06-07,Autonomous robot inspects in-service diesel st...,"HOUSTON Phillips 66, Square Robot Inc. and Ver...",https://www.proquest.com/newspapers/autonomous...,8601,"[[CLS] houston phillips 66, square robot inc. ...","[[0.4957221448421478, 0.008983639068901539, 0...."
8602,ProQuest,2.233474e+09,PSX,2019-06-03,United States/Mexico: Report: Trump's Tariff o...,US President Donald Trump's threats to tax Mex...,https://www.proquest.com/newspapers/united-sta...,8602,[[CLS] us president donald trump's threats to ...,"[[0.17639592289924622, 0.7728660106658936, 0.0..."


Merge chunk sentiments into text sentiment

In [ ]:
def average_sentiments(chunk_sentiment_list):
  sentiment_sum = [sum(x) for x in zip(*chunk_sentiment_list)]
  sentiment_avg = [x / len(chunk_sentiment_list) for x in sentiment_sum]
  return sentiment_avg

# Summarize Chunk Sentiments into an Article Sentiment
df['Sentiments']= df['Chunk Sentiments'].apply(average_sentiments)
# Split the 'Sentiments' column into three separate columns
df[['Neutral', 'Positive', 'Negative']] = pd.DataFrame(df['Sentiments'].tolist(), index=df.index)
df = df.drop('Sentiments', axis=1)
df = df.drop('Chunk Sentiments', axis=1)
df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index,Text Chunks,Neutral,Positive,Negative
0,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"[[CLS] stock report | may 16, 2024 | nysesymbo...",0.143564,0.413686,0.442750
1,Investment Research,NaN,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,1,"[[CLS] stock report | may 14, 2024 | nysesymbo...",0.164022,0.042301,0.793678
2,Investment Research,NaN,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,2,"[[CLS] stock report | may 11, 2024 | nysesymbo...",0.138396,0.089975,0.771629
3,Investment Research,NaN,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,3,"[[CLS] stock report | may 11, 2024 | nysesymbo...",0.047985,0.468556,0.483459
4,Investment Research,NaN,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,4,"[[CLS] stock report | may 07, 2024 | nysesymbo...",0.266492,0.181205,0.552303
...,...,...,...,...,...,...,...,...,...,...,...,...
8599,ProQuest,2.238654e+09,PSX,2019-06-12,"As the Price of Gas Tumbles, Even California I...",Gasoline prices have been falling fast in rece...,https://www.proquest.com/newspapers/as-price-g...,8599,[[CLS] gasoline prices have been falling fast ...,0.024151,0.501878,0.473971
8600,ProQuest,2.239478e+09,PSX,2019-06-10,"Phillips 66, Plains plan pipeline system",CUSHING Phillips 66 and Plains All American Pi...,https://www.proquest.com/newspapers/phillips-6...,8600,[[CLS] cushing phillips 66 and plains all amer...,0.097232,0.011336,0.891432
8601,ProQuest,2.238860e+09,PSX,2019-06-07,Autonomous robot inspects in-service diesel st...,"HOUSTON Phillips 66, Square Robot Inc. and Ver...",https://www.proquest.com/newspapers/autonomous...,8601,"[[CLS] houston phillips 66, square robot inc. ...",0.495722,0.008984,0.495294
8602,ProQuest,2.233474e+09,PSX,2019-06-03,United States/Mexico: Report: Trump's Tariff o...,US President Donald Trump's threats to tax Mex...,https://www.proquest.com/newspapers/united-sta...,8602,[[CLS] us president donald trump's threats to ...,0.098247,0.857687,0.044066


In [ ]:
#create chunk dataframe
df_chunk = df.explode('Text Chunks').reset_index(drop=True)
df_chunk = df_chunk.drop('Article Text', axis=1)
df_chunk['Ticker'] = df_chunk['Ticker'].fillna('')
#df_chunk = df_chunk.drop('Chunk Sentiments', axis=1)

# remove chunks from main dataframe
df = df.drop('Text Chunks', axis=1)

df_chunk

,Source,Unique_ID,Ticker,Date,Article Headline,URL,Article Index,Text Chunks,Neutral,Positive,Negative
0,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,NaN,0,"[CLS] stock report | may 16, 2024 | nysesymbol...",0.143564,0.413686,0.442750
1,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,NaN,0,"797 e 1, 804 e 7, 155 2024 1, 536 e 1, 705 e 1...",0.143564,0.413686,0.442750
2,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,NaN,0,". 12 - 1. 16 uin q1 2024, mro ’ s capex ( $ 60...",0.143564,0.413686,0.442750
3,Investment Research,NaN,MRO,2024-05-16,Marathon Oil Corporation,NaN,0,be relied as such. mro with revenue growth of ...,0.143564,0.413686,0.442750
4,Investment Research,NaN,EOG,2024-05-14,"EOG Resources, Inc.",NaN,1,"[CLS] stock report | may 14, 2024 | nysesymbol...",0.164021,0.042301,0.793678
...,...,...,...,...,...,...,...,...,...,...,...
26491,ProQuest,2.238860e+09,PSX,2019-06-07,Autonomous robot inspects in-service diesel st...,https://www.proquest.com/newspapers/autonomous...,8601,"[CLS] houston phillips 66, square robot inc. a...",0.495722,0.008984,0.495294
26492,ProQuest,2.233474e+09,PSX,2019-06-03,United States/Mexico: Report: Trump's Tariff o...,https://www.proquest.com/newspapers/united-sta...,8602,[CLS] us president donald trump's threats to t...,0.098247,0.857687,0.044066
26493,ProQuest,2.233474e+09,PSX,2019-06-03,United States/Mexico: Report: Trump's Tariff o...,https://www.proquest.com/newspapers/united-sta...,8602,to blend with lighter us supply to produce fue...,0.098247,0.857687,0.044066
26494,ProQuest,2.232567e+09,PSX,2019-05-31,Trump's Mexico Tariffs Could Hit Americans at ...,https://www.proquest.com/newspapers/trumps-mex...,8603,[CLS] president trump's proposed tariffs on me...,0.084698,0.835622,0.079681


Write to GitHub Repo

In [ ]:
# save csv locally
df.to_csv('/content/sample_data/sentiment_data.csv', index=True)
df_chunk.to_csv('/content/sample_data/sentiment_chunkdata.csv', index=True)

In [ ]:
#Copy CSV to GitHub Repository
!cp /content/sample_data/sentiment_data.csv /content/CVX_Rice_project/03_Sentiment_Analysis/
!cp /content/sample_data/sentiment_chunkdata.csv /content/CVX_Rice_project/03_Sentiment_Analysis/

In [ ]:
#Commit and Push Changes
!git config --global user.email "bw58@rice.edu"
%cd /content/CVX_Rice_project/03_Sentiment_Analysis/
!git add sentiment_data.csv
!git commit -m "Add sentiment_data.csv"
!git add sentiment_chunkdata.csv
!git commit -m "Add sentiment_chunkdata.csv"
!git push origin main

/content/CVX_Rice_project/03_Sentiment_Analysis
[main 770df72] Add sentiment_data.csv
 1 file changed, 384476 insertions(+), 204703 deletions(-)
[main 80183b3] Add sentiment_chunkdata.csv
 1 file changed, 26497 insertions(+), 14426 deletions(-)
Enumerating objects: 12, done.
Counting objects: 100% (12/12), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 7.29 MiB | 1.89 MiB/s, done.
Total 8 (delta 6), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (6/6), completed with 4 local objects.
To https://github.com/Kussil/CVX_Rice_project.git
   bfee45e..80183b3  main -> main


In [ ]:
#!rm -rf /content/CVX_Rice_project/